<a href="https://colab.research.google.com/github/armandossrecife/my_validation2/blob/main/analise_issues_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piloto 1 - Análise dos issues do Cassandra

2a. Rodada (pós críicas)

Cenário: Issues que aparecem nos commits com classes críticas

Commits analisados: 29230

Issues analisados: 18635

Faixa de análise: from 2/03/2009 to 4/10/2023

Repositório Apache Cassandra: https://github.com/apache/cassandra

Issue Tracker: https://issues.apache.org/jira/projects/CASSANDRA

# Processo de coleta, análise e interpretação dos resultados


**1.   Coleta dos dados**

Definição da nossa população de interesse

- população de commits: todos os commits do repositório "Apache Cassandra" no GitHub entre as datas especificadas.
- população de issues: todos os issues do Issue Tracker do "Apache Cassandra" entre as datas especificadas

**2.   Seleção da amostra**

Seleção de uma parte dos issues suspeitos que serão analisados.

- uma amostra aleatória de "issues suspeitos" em vez de toda a população de issues, devido a necessidade de realizar inspeção manual nos issues e esta tarefa "consome muito tempo".
- Tempo médio de inspeção manual de issues: 40 issues por dia.
- "issues suspeitos" são os issues que aparecem nos commits com arquivos críticos.

**3.   Cálculo da amostra**

Baseado em uma variável aleatória contínua normal do [ScyPY](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)

- Baseado nos parâmetros: confidence_level = 0.95, margin_of_error = 0.05, population_proportion = 0.5 e population_size = len(quantidade_de_issues_suspeitos)
- sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
- norm.ppf(): variável aleatória contínua normal

**4. Análise estatística**

Após a coleta de dados, vamos realizar as análises estatísticas dos issues para checar se os issues suspeitos indicam impacto arquitetural.


**5. Visualização dos dados**

Vamos criar os gráficos e visualizações sobre os issues suspeitos

**6. Interpretação dos resultados**

Vamos interpretar os resultados das análises estatísticas e tirar nossas conclusões sobre os issues suspeitos.

## Perguntas

Primeira parte:

Q1. Quais são as classes críticas?

Q2. Quais são os commits que possuem classes críticas?

Q3. Quais são os issues que aparecem nos commits que possuem classes críticas?

Segunda parte:

Q4. Os issues da Q3 possuem impacto arquitetural?

Q5. Quanto (%) dos issues da Q3 possuem impacto arquitetural?

### Funções de apoio

In [112]:
import pandas as pd
import regex as re
import scipy.stats as stats
import random
import tqdm
import os

def find_all_issues_cassandra(input_string: str) -> list[str]:
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r'(CASSANDRA-\d+)', input_string)
    set_matches = set(matches)
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    print('Creating a new text file...')
    print(f'Create file {filename} in {dir_name}')
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro: {str(ex)}')


### Importa o dataset inicial

In [113]:
!wget https://raw.githubusercontent.com/armandossrecife/my_validation2/main/commits_arquivos_criticos.xlsx
!wget https://raw.githubusercontent.com/armandossrecife/my_validation2/main/issues_in_commits_arquivos_criticos.xlsx
!wget https://raw.githubusercontent.com/armandossrecife/my_validation2/main/commits_arquivos_critico_e_issues.txt

--2023-10-26 20:37:21--  https://raw.githubusercontent.com/armandossrecife/my_validation2/main/commits_arquivos_criticos.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13233731 (13M) [application/octet-stream]
Saving to: ‘commits_arquivos_criticos.xlsx’

commits_arquivos_cr 100%[===================>]  12.62M  --.-KB/s    in 0.1s    

2023-10-26 20:37:21 (99.8 MB/s) - ‘commits_arquivos_criticos.xlsx’ saved [13233731/13233731]

--2023-10-26 20:37:21--  https://raw.githubusercontent.com/armandossrecife/my_validation2/main/issues_in_commits_arquivos_criticos.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.19

# Primeira Parte

## Q1. Classes críticas

From [ATDCodeAnalyzer](https://github.com/mining-software-repositories/cassandra)

São classes impactadas por ATD identificadas pelo ATDCodeAnalyzer

In [114]:
lista_classes_criticas = ['StorageService.java', 'ColumnFamilyStore.java', 'DatabaseDescriptor.java', 'CompactionManager.java', 'StorageProxy.java', 'SSTableReader.java', 'Config.java', 'CassandraDaemon.java', 'SelectStatement.java', 'SinglePartitionReadCommand.java', 'NodeProbe.java', 'MessagingService.java']
print(len(lista_classes_criticas), lista_classes_criticas)

12 ['StorageService.java', 'ColumnFamilyStore.java', 'DatabaseDescriptor.java', 'CompactionManager.java', 'StorageProxy.java', 'SSTableReader.java', 'Config.java', 'CassandraDaemon.java', 'SelectStatement.java', 'SinglePartitionReadCommand.java', 'NodeProbe.java', 'MessagingService.java']


## Q2. Commits com classes críticas

From [My_validation](https://www.github.com/armandossrecife/my_validation)

São os commits que possuem pelo menos uma classe crítica em seu registro

In [115]:
df_commits_classes_criticas = pd.read_excel('commits_arquivos_criticos.xlsx')
df_commits_classes_criticas

,hahs,msg,m_files,date,lines,files,diff_files,diff_modified_files
0,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,git-svn-id: https://svn.apache.org/repos/asf/i...,"['DatabaseDescriptor.java', 'ColumnFamilyStore...",2/3/2009,74502,344,"[\n {\n ""DatabaseDescriptor.java"": ""...","[\n ""Binary files /dev/null and b/lib/JSAP-..."
1,cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,Add build script and make sure it works\n\ngit...,['StorageService.java'],2/3/2009,156,4,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -0,0 +1,133 @@\n+<?xml version=\""1...."
2,5bea7e1772e9fd042b10abebb1e0c78579942b26,Removed references to JDK7 specific things.\n\...,['ColumnFamilyStore.java'],20/3/2009,82,2,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -20,7 +20,6 @@ package org.apache.c..."
3,f4f3ca50d84e165a909dca5608babb32df199f44,rename serializer2 -> serializerWithIndexers\n...,['ColumnFamilyStore.java'],27/3/2009,26,4,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -70,7 +70,7 @@ public final class C..."
4,39d3f833f9c4e154eb157336f5e7b2a1ce9a79a3,add SuperColumn support to forceFlush. split ...,['ColumnFamilyStore.java'],27/3/2009,89,5,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -395,11 +395,11 @@ public class Col..."
...,...,...,...,...,...,...,...,...
4517,6708499e4c930f7733894e6076b925fb151d9251,Remove metrics-reporter-config dependency\n\np...,['CassandraDaemon.java'],12/9/2023,105,10,"[\n {\n ""CassandraDaemon.java"": ""@@ ...","[\n ""@@ -43,7 +43,6 @@\n <..."
4518,20d80118ac02d853d501e3f5c24a63a12d4010f5,Add the ability to disable bulk loading of SST...,['Config.java'],19/9/2023,289,11,"[\n {\n ""Config.java"": ""@@ -860,6 +8...","[\n ""@@ -1,4 +1,5 @@\n 5.1\n+ * Add the abi..."
4519,d16e8d3653dce8ed767a040c06dbaabc47a9b474,Do not create sstable files before registering...,['CompactionManager.java'],25/9/2023,724,18,"[\n {\n ""CompactionManager.java"": ""@...","[\n ""@@ -1,4 +1,5 @@\n 5.0-alpha2\n+ * Do n..."
4520,b9586501a6b6cdfe465302448018785652c9b966,Internode legacy SSL storage port certificate ...,['MessagingService.java'],25/9/2023,237,13,"[\n {\n ""MessagingService.java"": ""@@...","[\n ""@@ -1,4 +1,5 @@\n 4.1.4\n+ * Internode..."


In [116]:
print(f'Qtd de commits com classes críticas: {len(df_commits_classes_criticas)}')

Qtd de commits com classes críticas: 4522


## Q3. Issues que aparecem nos commits com classes críticas

From [My_validation](https://www.github.com/armandossrecife/my_validation)

São issues "suspeitos" de terem impacto arquitetural

In [117]:
df_issues_commits_classes_criticas = pd.read_excel('issues_in_commits_arquivos_criticos.xlsx')
df_issues_commits_classes_criticas

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments
0,CASSANDRA-261,get_slice needs offset + limit,Bug,Resolved,Normal,Right now get_slice does not allow you to prov...,"['Me, explaining new get_slice: so now you wou..."
1,CASSANDRA-259,LRU cache for key positions,New Feature,Resolved,Normal,"add cache like the old touch cache, but workin...","[""LinkedHashMap is a nonstarter though. This ..."
2,CASSANDRA-254,clean up sstable constructors,Improvement,Resolved,Normal,NaN,"['+1', 'committed', ""Integrated in Cassandra #..."
3,CASSANDRA-251,Restore ColumnSort=Time on Super Columnfamilies,Improvement,Resolved,Normal,Setting ColumnSort=Time on a Super Columnfamil...,"['Sorry, I got a little rattled by the ""oh cra..."
4,CASSANDRA-248,Replace factory method based on explicit locks...,Improvement,Resolved,Normal,Replace factory method based on explicit locks...,"['Patch submitted.', 'This is double-checked l..."
...,...,...,...,...,...,...,...
2907,CASSANDRA-18504,"Added support for type VECTOR<type, dimension>",Improvement,Resolved,Normal,"Based off several mailing list threads (see ""[...",['having a hard time fixing CQL3TypeLiteralTes...
2908,CASSANDRA-18500,Add guardrail for partition size,New Feature,Resolved,Normal,"Add a guardrail for max partition size, for ex...","[""The proposed patch adds a new guardrail for ..."
2909,CASSANDRA-18816,Add support for repair coordinator to retry me...,Improvement,Resolved,Normal,"Now that CASSANDRA-15399 is in, most of the re...",['scope slightly increased due to bugs found b...
2910,CASSANDRA-18803,Refactor validation logic in StorageService.re...,Bug,Resolved,Normal,This is a follow-up ticket of CASSANDRA-14319,['[~aweisberg] would you mind to take a look? ...


In [118]:
print('Quantidade de issues nos commits com classes críticas: ')
print(len(df_issues_commits_classes_criticas))

Quantidade de issues nos commits com classes críticas: 
2912


### Commits com classes críticas e issues registrados

In [119]:
# Carrega os commits com arquivos críticos e issues
lista_commits_arquivos_critico_e_issues = []
with open('commits_arquivos_critico_e_issues.txt', mode='r') as f_temp:
  for linha in f_temp:
    linha = linha.strip()
    lista_commits_arquivos_critico_e_issues.append(linha)

### Filtra apenas os commits com classes críticas e issues registrados

In [120]:
filtered_df = df_commits_classes_criticas[df_commits_classes_criticas['hahs'].isin(lista_commits_arquivos_critico_e_issues)]
filtered_df

,hahs,msg,m_files,date,lines,files,diff_files,diff_modified_files
68,91afe8b6e9e9f0a808261b72c8532841b871ba45,move *Partitioner from cassandra.service to ca...,['StorageService.java'],27/4/2009,31,6,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -44,7 +44,7 @@ import org.apache.ca..."
69,63a748ccf9ab5192456baca9dd90b4c0cf2a2038,r/m unused code\npatch by jbellis; reviewed by...,['StorageService.java'],27/4/2009,72,2,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -71,40 +71,7 @@ public class RackUn..."
70,907d92e21b8ce79e6a224933764e3b7215a94454,replace BigInteger tokens with BigIntegerToken...,"['ColumnFamilyStore.java', 'StorageService.java']",27/4/2009,1090,31,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -43,6 +43,7 @@ import org.apache.lo..."
71,ec7f04b4df62b87231d9cacca754a209bd8fcc59,r/m StorageService.token in favor of explicitl...,"['ColumnFamilyStore.java', 'StorageService.java']",27/4/2009,101,8,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -1113,7 +1113,7 @@ public class Col..."
72,296e15b63cdb678a12db004e29359eaf8c14c6c5,add test for ReplicaPlacementStrategy covering...,['StorageService.java'],27/4/2009,77,3,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -2,7 +2,7 @@ package org.apache.cas..."
...,...,...,...,...,...,...,...,...
4517,6708499e4c930f7733894e6076b925fb151d9251,Remove metrics-reporter-config dependency\n\np...,['CassandraDaemon.java'],12/9/2023,105,10,"[\n {\n ""CassandraDaemon.java"": ""@@ ...","[\n ""@@ -43,7 +43,6 @@\n <..."
4518,20d80118ac02d853d501e3f5c24a63a12d4010f5,Add the ability to disable bulk loading of SST...,['Config.java'],19/9/2023,289,11,"[\n {\n ""Config.java"": ""@@ -860,6 +8...","[\n ""@@ -1,4 +1,5 @@\n 5.1\n+ * Add the abi..."
4519,d16e8d3653dce8ed767a040c06dbaabc47a9b474,Do not create sstable files before registering...,['CompactionManager.java'],25/9/2023,724,18,"[\n {\n ""CompactionManager.java"": ""@...","[\n ""@@ -1,4 +1,5 @@\n 5.0-alpha2\n+ * Do n..."
4520,b9586501a6b6cdfe465302448018785652c9b966,Internode legacy SSL storage port certificate ...,['MessagingService.java'],25/9/2023,237,13,"[\n {\n ""MessagingService.java"": ""@@...","[\n ""@@ -1,4 +1,5 @@\n 4.1.4\n+ * Internode..."


In [121]:
print(f'Qtd de commits com classes críticas e issues registrados nas mensagens: {len(filtered_df)}')

Qtd de commits com classes críticas e issues registrados nas mensagens: 3458


### Issues registrados nas mensagnes dos commits com classes críticas

Cria um dataframe integrando informações de commits e issues

In [122]:
relacao_commit_lista_issues = []
for index in filtered_df.index:
  if len(find_all_issues_cassandra(input_string=filtered_df.msg[index]))>0:
    #print(filtered_df.msg[index])
    elemento = filtered_df.hahs[index], find_all_issues_cassandra(input_string=filtered_df.msg[index])
    relacao_commit_lista_issues.append(elemento)

l_hash, l_issues = [], []
for each in relacao_commit_lista_issues:
  l_hash.append(each[0])
  l_issues.append(each[1])

dict_relacao_commit_issues = {
  'hahs':l_hash,
  'lista_issues':l_issues
}

df_relacao_commit_issues = pd.DataFrame(dict_relacao_commit_issues)
df_relacao_commit_issues

,hahs,lista_issues
0,91afe8b6e9e9f0a808261b72c8532841b871ba45,[CASSANDRA-65]
1,63a748ccf9ab5192456baca9dd90b4c0cf2a2038,[CASSANDRA-65]
2,907d92e21b8ce79e6a224933764e3b7215a94454,[CASSANDRA-65]
3,ec7f04b4df62b87231d9cacca754a209bd8fcc59,[CASSANDRA-65]
4,296e15b63cdb678a12db004e29359eaf8c14c6c5,[CASSANDRA-65]
...,...,...
3453,6708499e4c930f7733894e6076b925fb151d9251,[CASSANDRA-18743]
3454,20d80118ac02d853d501e3f5c24a63a12d4010f5,[CASSANDRA-18781]
3455,d16e8d3653dce8ed767a040c06dbaabc47a9b474,[CASSANDRA-18737]
3456,b9586501a6b6cdfe465302448018785652c9b966,[CASSANDRA-18681]


In [123]:
df_final = pd.merge(filtered_df, df_relacao_commit_issues, how='inner')
df_final = df_final[['hahs', 'lista_issues', 'msg', 'm_files', 'date', 'lines', 'files', 'diff_files','diff_modified_files']]
df_final

,hahs,lista_issues,msg,m_files,date,lines,files,diff_files,diff_modified_files
0,91afe8b6e9e9f0a808261b72c8532841b871ba45,[CASSANDRA-65],move *Partitioner from cassandra.service to ca...,['StorageService.java'],27/4/2009,31,6,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -44,7 +44,7 @@ import org.apache.ca..."
1,63a748ccf9ab5192456baca9dd90b4c0cf2a2038,[CASSANDRA-65],r/m unused code\npatch by jbellis; reviewed by...,['StorageService.java'],27/4/2009,72,2,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -71,40 +71,7 @@ public class RackUn..."
2,907d92e21b8ce79e6a224933764e3b7215a94454,[CASSANDRA-65],replace BigInteger tokens with BigIntegerToken...,"['ColumnFamilyStore.java', 'StorageService.java']",27/4/2009,1090,31,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -43,6 +43,7 @@ import org.apache.lo..."
3,ec7f04b4df62b87231d9cacca754a209bd8fcc59,[CASSANDRA-65],r/m StorageService.token in favor of explicitl...,"['ColumnFamilyStore.java', 'StorageService.java']",27/4/2009,101,8,"[\n {\n ""ColumnFamilyStore.java"": ""@...","[\n ""@@ -1113,7 +1113,7 @@ public class Col..."
4,296e15b63cdb678a12db004e29359eaf8c14c6c5,[CASSANDRA-65],add test for ReplicaPlacementStrategy covering...,['StorageService.java'],27/4/2009,77,3,"[\n {\n ""StorageService.java"": ""@@ -...","[\n ""@@ -2,7 +2,7 @@ package org.apache.cas..."
...,...,...,...,...,...,...,...,...,...
3453,6708499e4c930f7733894e6076b925fb151d9251,[CASSANDRA-18743],Remove metrics-reporter-config dependency\n\np...,['CassandraDaemon.java'],12/9/2023,105,10,"[\n {\n ""CassandraDaemon.java"": ""@@ ...","[\n ""@@ -43,7 +43,6 @@\n <..."
3454,20d80118ac02d853d501e3f5c24a63a12d4010f5,[CASSANDRA-18781],Add the ability to disable bulk loading of SST...,['Config.java'],19/9/2023,289,11,"[\n {\n ""Config.java"": ""@@ -860,6 +8...","[\n ""@@ -1,4 +1,5 @@\n 5.1\n+ * Add the abi..."
3455,d16e8d3653dce8ed767a040c06dbaabc47a9b474,[CASSANDRA-18737],Do not create sstable files before registering...,['CompactionManager.java'],25/9/2023,724,18,"[\n {\n ""CompactionManager.java"": ""@...","[\n ""@@ -1,4 +1,5 @@\n 5.0-alpha2\n+ * Do n..."
3456,b9586501a6b6cdfe465302448018785652c9b966,[CASSANDRA-18681],Internode legacy SSL storage port certificate ...,['MessagingService.java'],25/9/2023,237,13,"[\n {\n ""MessagingService.java"": ""@@...","[\n ""@@ -1,4 +1,5 @@\n 4.1.4\n+ * Internode..."


# Segunda Parte

## Seleção da amostra de issues

In [124]:
confidence_level = 0.95
margin_of_error = 0.05
population_proportion = 0.5
population_size = len(df_issues_commits_classes_criticas)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 2912
Sample size para inspeção: 339


In [125]:
# Seleciona randomicamente os issues para inspeção
lista_issues_inspecao = []
dict_issues_para_inspecao = {}
list_issue_key = df_issues_commits_classes_criticas.issue_key.to_list()
sample_issues = random.choices(list_issue_key, k=sample_size)
dict_issues_para_inspecao['26/10/2023'] = sample_issues
print(f'{len(sample_issues)} para inspeção manual')

339 para inspeção manual


In [126]:
print('Issues guardados em 26/10/2023')
with open('issues_inspecao_26102023.txt', mode='w') as f_temp:
  for v in dict_issues_para_inspecao['26/10/2023']:
    elemento = v + ','
    f_temp.write(elemento)
print('Relação de Issues salvos em 26/10/2023 para inspeção.')

Issues guardados em 26/10/2023
Relação de Issues salvos em 26/10/2023 para inspeção.


### Gera os arquivos .txt da amostra de issues suspeitos

In [127]:
# Gera os arquivos .txt de cada issue selecionado para inspeção
my_dir_name = 'my_issues'
if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

total_of_issues = df_issues_commits_classes_criticas.shape[0]
for index in tqdm.tqdm(df_issues_commits_classes_criticas.index, total=total_of_issues, desc='Analyzing issues'):
  if df_issues_commits_classes_criticas.issue_key[index] in sample_issues:
    create_new_file(filename=df_issues_commits_classes_criticas.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_commits_classes_criticas.issue_type[index], summary=df_issues_commits_classes_criticas.summary[index], description=df_issues_commits_classes_criticas.description[index], status=df_issues_commits_classes_criticas.issue_status[index], comments=df_issues_commits_classes_criticas.comments[index])

Analyzing issues:  18%|█▊        | 527/2912 [00:00<00:00, 5234.14it/s]

Creating a new text file...
Create file CASSANDRA-254 in my_issues
Erro: object of type 'float' has no len()
Creating a new text file...
Create file CASSANDRA-211 in my_issues
File my_issues/CASSANDRA-211 created with success!
Creating a new text file...
Create file CASSANDRA-208 in my_issues
File my_issues/CASSANDRA-208 created with success!
Creating a new text file...
Create file CASSANDRA-201 in my_issues
File my_issues/CASSANDRA-201 created with success!
Creating a new text file...
Create file CASSANDRA-193 in my_issues
File my_issues/CASSANDRA-193 created with success!
Creating a new text file...
Create file CASSANDRA-185 in my_issues
File my_issues/CASSANDRA-185 created with success!
Creating a new text file...
Create file CASSANDRA-157 in my_issues
File my_issues/CASSANDRA-157 created with success!
Creating a new text file...
Create file CASSANDRA-149 in my_issues
Erro: object of type 'float' has no len()
Creating a new text file...
Create file CASSANDRA-120 in my_issues
File my

Analyzing issues:  36%|███▌      | 1054/2912 [00:00<00:00, 5246.92it/s]


File my_issues/CASSANDRA-4125 created with success!
Creating a new text file...
Create file CASSANDRA-4079 in my_issues
File my_issues/CASSANDRA-4079 created with success!
Creating a new text file...
Create file CASSANDRA-4076 in my_issues
File my_issues/CASSANDRA-4076 created with success!
Creating a new text file...
Create file CASSANDRA-4004 in my_issues
File my_issues/CASSANDRA-4004 created with success!
Creating a new text file...
Create file CASSANDRA-4764 in my_issues
File my_issues/CASSANDRA-4764 created with success!
Creating a new text file...
Create file CASSANDRA-4750 in my_issues
File my_issues/CASSANDRA-4750 created with success!
Creating a new text file...
Create file CASSANDRA-4723 in my_issues
File my_issues/CASSANDRA-4723 created with success!
Creating a new text file...
Create file CASSANDRA-4710 in my_issues


Analyzing issues:  56%|█████▌    | 1630/2912 [00:00<00:00, 5464.02it/s]

File my_issues/CASSANDRA-4710 created with success!
Creating a new text file...
Create file CASSANDRA-4655 in my_issues
File my_issues/CASSANDRA-4655 created with success!
Creating a new text file...
Create file CASSANDRA-4647 in my_issues
File my_issues/CASSANDRA-4647 created with success!
Creating a new text file...
Create file CASSANDRA-4559 in my_issues
File my_issues/CASSANDRA-4559 created with success!
Creating a new text file...
Create file CASSANDRA-4532 in my_issues
File my_issues/CASSANDRA-4532 created with success!
Creating a new text file...
Create file CASSANDRA-4495 in my_issues
File my_issues/CASSANDRA-4495 created with success!
Creating a new text file...
Create file CASSANDRA-4368 in my_issues
File my_issues/CASSANDRA-4368 created with success!
Creating a new text file...
Create file CASSANDRA-5160 in my_issues
File my_issues/CASSANDRA-5160 created with success!
Creating a new text file...
Create file CASSANDRA-5112 in my_issues
File my_issues/CASSANDRA-5112 created wi

Analyzing issues:  76%|███████▌  | 2201/2912 [00:00<00:00, 5509.87it/s]


File my_issues/CASSANDRA-10661 created with success!
Creating a new text file...
Create file CASSANDRA-10652 in my_issues
File my_issues/CASSANDRA-10652 created with success!
Creating a new text file...
Create file CASSANDRA-10572 in my_issues
File my_issues/CASSANDRA-10572 created with success!
Creating a new text file...
Create file CASSANDRA-10571 in my_issues
File my_issues/CASSANDRA-10571 created with success!
Creating a new text file...
Create file CASSANDRA-10512 in my_issues
File my_issues/CASSANDRA-10512 created with success!
Creating a new text file...
Create file CASSANDRA-10413 in my_issues
File my_issues/CASSANDRA-10413 created with success!
Creating a new text file...
Create file CASSANDRA-11840 in my_issues
Erro: object of type 'float' has no len()
Creating a new text file...
Create file CASSANDRA-11217 in my_issues
File my_issues/CASSANDRA-11217 created with success!
Creating a new text file...
Create file CASSANDRA-11172 in my_issues
File my_issues/CASSANDRA-11172 cre

Analyzing issues: 100%|██████████| 2912/2912 [00:00<00:00, 5201.90it/s]

Creating a new text file...
Create file CASSANDRA-12373 in my_issues
File my_issues/CASSANDRA-12373 created with success!
Creating a new text file...
Create file CASSANDRA-12248 in my_issues
File my_issues/CASSANDRA-12248 created with success!
Creating a new text file...
Create file CASSANDRA-12197 in my_issues
File my_issues/CASSANDRA-12197 created with success!
Creating a new text file...
Create file CASSANDRA-12148 in my_issues
File my_issues/CASSANDRA-12148 created with success!
Creating a new text file...
Create file CASSANDRA-12143 in my_issues
File my_issues/CASSANDRA-12143 created with success!
Creating a new text file...
Create file CASSANDRA-12127 in my_issues
File my_issues/CASSANDRA-12127 created with success!
Creating a new text file...
Create file CASSANDRA-12071 in my_issues
File my_issues/CASSANDRA-12071 created with success!
Creating a new text file...
Create file CASSANDRA-12043 in my_issues
File my_issues/CASSANDRA-12043 created with success!
Creating a new text file

In [128]:
!zip -r my_issues.zip my_issues

  adding: my_issues/ (stored 0%)
  adding: my_issues/CASSANDRA-18617 (deflated 60%)
  adding: my_issues/CASSANDRA-1489 (deflated 52%)
  adding: my_issues/CASSANDRA-12781 (deflated 72%)
  adding: my_issues/CASSANDRA-6551 (deflated 46%)
  adding: my_issues/CASSANDRA-5506 (deflated 47%)
  adding: my_issues/CASSANDRA-9945 (deflated 52%)
  adding: my_issues/CASSANDRA-18441 (deflated 58%)
  adding: my_issues/CASSANDRA-17733 (deflated 57%)
  adding: my_issues/CASSANDRA-15018 (deflated 42%)
  adding: my_issues/CASSANDRA-10783 (deflated 54%)
  adding: my_issues/CASSANDRA-4723 (deflated 56%)
  adding: my_issues/CASSANDRA-338 (deflated 55%)
  adding: my_issues/CASSANDRA-8398 (deflated 54%)
  adding: my_issues/CASSANDRA-1334 (deflated 44%)
  adding: my_issues/CASSANDRA-304 (deflated 29%)
  adding: my_issues/CASSANDRA-12768 (deflated 56%)
  adding: my_issues/CASSANDRA-424 (stored 0%)
  adding: my_issues/CASSANDRA-18062 (deflated 55%)
  adding: my_issues/CASSANDRA-7136 (deflated 53%)
  adding: my_is

In [129]:
#!rm -rf my_issues

## Q4. Os issues da Q3 possuem impacto arquitetural?

Em andamento...

## Q5. Quanto (%) dos issues da Q3 possuem impacto arquitetural?

Em andamento...
